## Text Generation with RNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#open text file
raw_text = open("war-peace.txt","r").read()
print(raw_text[:100])
raw_text = raw_text.lower()
print(raw_text[:100])

"Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if
"well, prince, so genoa and lucca are now just family estates of the
buonapartes. but i warn you, if


In [3]:
#Counting chars
n_chars = len(raw_text)
print('Total characters: {}'.format(n_chars))

Total characters: 3196212


In [4]:
#Getting unique characters and vocabulary size
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print('Total vocabulary (unique characters): {}'.format(n_vocab))
print(chars)

Total vocabulary (unique characters): 56
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ä', 'é', 'ê']



#### برای این که متن هارا به عدد تبدیل کنیم، یک دیکشنری میسازیم که به هر کاراکتر یک عدد نسبت میدهیم
#### برای این که از روی اعداد بتوانیم متن را بسازیم نیز دیکشنری برعکس بالا میسازیم

In [5]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))  #inverse
print(char_to_index)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '=': 24, '?': 25, 'a': 26, 'b': 27, 'c': 28, 'd': 29, 'e': 30, 'f': 31, 'g': 32, 'h': 33, 'i': 34, 'j': 35, 'k': 36, 'l': 37, 'm': 38, 'n': 39, 'o': 40, 'p': 41, 'q': 42, 'r': 43, 's': 44, 't': 45, 'u': 46, 'v': 47, 'w': 48, 'x': 49, 'y': 50, 'z': 51, 'à': 52, 'ä': 53, 'é': 54, 'ê': 55}


In [45]:
seq_length = 100 #sequence lenth
n_seq = int(n_chars / seq_length) # Count of sequence
print(n_seq)

X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))

31962


##### توجه داشته باشید که طول دنباله از نظر شکل اینگونه است:
### (تعداد نمونه، طول دنباله، ابعاد ویژگی)

![<caption>](pic1.jpg)

In [46]:
for i in range(n_seq):
    x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
    x_sequence_ohe = np.zeros((seq_length, n_vocab))
    for j in range(seq_length):
        char = x_sequence[j]
        index = char_to_index[char]
        x_sequence_ohe[j][index] = 1.
        X[i] = x_sequence_ohe
        y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1]
        y_sequence_ohe = np.zeros((seq_length, n_vocab))
    for j in range(seq_length):
        char = y_sequence[j]
        index = char_to_index[char]
        y_sequence_ohe[j][index] = 1.
        Y[i] = y_sequence_ohe

In [57]:
print(raw_text[:100])
print(X[0][1])

"well, prince, so genoa and lucca are now just family estates of the
buonapartes. but i warn you, if
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0.]


In [60]:
#firt char-->" 
#second char--> w
test = X[0][1]
index_to_char[np.argmax(test)]

'w'

In [61]:
#first char--> w
test = Y[0,0]
index_to_char[np.argmax(test)]

'w'

In [62]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras import optimizers
from tensorflow import keras

In [63]:
n_layer=2

model = Sequential()
model.add(SimpleRNN(256, activation='relu', input_shape=(None,
n_vocab), return_sequences=True))
model.add(Dropout(0.3))
for i in range(n_layer - 1):
    model.add(SimpleRNN(256, activation='relu',
    return_sequences=True))
    model.add(Dropout(0.3))
model.add(TimeDistributed(Dense(n_vocab)))
model.add(Activation('softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9,
epsilon=1e-08, decay=0.0)

model.compile(loss= "categorical_crossentropy", optimizer=optimizer)

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 256)         80128     
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 256)         131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
time_distributed (TimeDistri (None, None, 56)          14392     
_________________________________________________________________
activation (Activation)      (None, None, 56)          0         
Total params: 225,848
Trainable params: 225,848
Non-trainable params: 0
__________________________________________________

In [65]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
filepath="weights/weights_layer_%d_hidden_%d_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1,
save_best_only=True, mode='min')

early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50,
verbose=1, mode='min')

In [20]:
def generate_text(model, gen_length, n_vocab, index_to_char):
    """Generating text using the RNN model
    @param model: current RNN model
    @param gen_length: number of characters we want to generate
    @param n_vocab: number of unique characters
    @param index_to_char: index to character mapping
    @return:"""
    # Start with a randomly picked character
    index = np.random.randint(n_vocab)
    y_char = [index_to_char[index]]
    X = np.zeros((1, gen_length, n_vocab))
    for i in range(gen_length):
        X[0, i, index] = 1.
        indices = np.argmax(model.predict(X[:, max(0, i - 99):i + 1, :])[0], 1)
        index = indices[-1]
        y_char.append(index_to_char[index])
        return ('').join(y_char)

In [21]:
class ResultChecker(Callback):
    def __init__(self, model, N, gen_length):
        self.model = model
        self.N = N
        self.gen_length = gen_length
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.N == 0:
            result = generate_text(self.model, self.gen_length, n_vocab,
            index_to_char)
            print('\nMy War and Peace:\n' + result)

In [25]:
model.fit(X, Y, batch_size=1, verbose=1, epochs=200,
    callbacks=[checkpoint, early_stop])

Epoch 1/10
320/320 [==============================] - ETA: 0s - loss: 1.9909
My War and Peace:
é 

Epoch 00001: loss improved from inf to 1.99089, saving model to weights\weights_layer_%d_hidden_%d_epoch_001_loss_1.9909.hdf5
320/320 [==============================] - 678s 2s/step - loss: 1.9909
Epoch 2/10
 15/320 [>.............................] - ETA: 10:17 - loss: 1.6125

KeyboardInterrupt: 